In [1]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import anndata as ad
import numpy as np
import collections

/Users/yijwang-admin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# read in data
with open("../../../Data/ROSMAP_blood_gene_expression_monocyte_ACTL_04172023.csv") as your_data:
    adata_rosmap = ad.read_csv(your_data, delimiter=',')
adata_rosmap = adata_rosmap.T
    
with open("../../../Data/ANMerge_blood_rna_gene_expr_processed_XH_10112023.csv") as your_data:
    adata_adni = ad.read_csv(your_data, delimiter=',')
adata_adni = adata_adni.T
    
# read in label
Blood_Info = pd.read_csv("../../../Data/ROSMAP_blood_clinical_monocyte_ACTL_04172023.csv",header=0, index_col=0)

# read in label
Blood_Adni_Info = pd.read_csv("../../../Data/ANMerge_NewModel_NewLabel_Clinical_Info.csv",header=0, index_col=0)

# read in projection from Monocle3
ADNI_Transferred_Label = pd.read_csv("../../../Data/Monocle_LabelTransfer_ROSMAP_ANMerge_Braak.csv")

In [9]:
SID = list(ADNI_Transferred_Label['X'])
PreLabel = list(ADNI_Transferred_Label['braak_fix'])
Mapping_ADNIID_PreLabel = dict(zip(SID, PreLabel))

In [10]:
#Mapping_ADNIID_PreLabel

In [11]:
ADNI_Transferred_Label

,Unnamed: 0,X,Label,Gender,Age,DX,Site,Batch,OnsetAge,OnsetPattern,Size_Factor,data_set,braak_xfr,braak_fix
0,1,DCR00001_1,Low-NFT AD,Female,80.0,AD,DCR,2,80.0,Sudden,0.995368,query,5.0,5
1,2,DCR00004_1,Typical AD,Male,78.0,MCI,DCR,2,inf,NaN,0.998300,query,NaN,5
2,3,DCR00009_1,Asym AD,Female,69.0,CTL,DCR,2,inf,NaN,1.001917,query,5.0,5
3,4,DCR00013_1,Asym AD,Male,82.0,CTL,DCR,2,inf,NaN,1.002106,query,5.0,5
4,5,DCR00014_1,Control,Female,75.0,CTL,DCR,2,inf,NaN,0.997070,query,5.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,687,TLSMCI607_1,Asym AD,Female,72.0,MCI,Toulouse,1,69.0,NaN,0.999104,query,5.0,5
687,688,TLSMCI608_1,Asym AD,Male,73.0,MCI,Toulouse,1,71.0,NaN,0.994493,query,5.0,5
688,689,TLSMCI609_1,Typical AD,Female,63.0,MCI,Toulouse,2,53.0,NaN,1.004423,query,5.0,5
689,690,TLSMCI611_1,Asym AD,Female,78.0,MCI,Toulouse,2,77.0,NaN,1.002816,query,5.0,5


In [12]:
Blood_Adni_Info

,Gender,Age,DX,Site,Batch,OnsetAge,OnsetPattern,MMSE,CDRSOB,CDRTotal,braaksc_trans,NewLabel
DCR00001_1,Female,80.0,AD,DCR,2,80.0,Sudden,21.0,5.0,1.0,4.0,Typical AD
DCR00004_1,Male,78.0,MCI,DCR,2,inf,NaN,27.0,0.5,0.5,2.0,Others
DCR00009_1,Female,69.0,CTL,DCR,2,inf,NaN,30.0,0.0,0.0,2.0,Control
DCR00013_1,Male,82.0,CTL,DCR,2,inf,NaN,30.0,0.0,0.0,4.0,Others
DCR00014_1,Female,75.0,CTL,DCR,2,inf,NaN,30.0,inf,inf,5.0,Others
...,...,...,...,...,...,...,...,...,...,...,...,...
TLSMCI607_1,Female,72.0,MCI,Toulouse,1,69.0,NaN,29.0,1.0,0.5,5.0,Asym AD
TLSMCI608_1,Male,73.0,MCI,Toulouse,1,71.0,NaN,28.0,1.0,0.5,4.0,Asym AD
TLSMCI609_1,Female,63.0,MCI,Toulouse,2,53.0,NaN,28.0,0.5,0.5,4.0,Asym AD
TLSMCI611_1,Female,78.0,MCI,Toulouse,2,77.0,NaN,30.0,1.0,0.5,4.0,Asym AD


In [14]:
SID = list(ADNI_Transferred_Label.X)
Braak_trans = list(ADNI_Transferred_Label.braak_fix)

Mapping_SID_Braak = dict(zip(SID, Braak_trans))

SID = list(Blood_Adni_Info.index)
DX = list(Blood_Adni_Info.DX)
CDRSB = list(Blood_Adni_Info.CDRSOB)

Mapping_SID_DX = dict(zip(SID, DX))
Mapping_SID_CDRSB = dict(zip(SID, CDRSB))


In [15]:
SID = list(adata_adni.obs.index)

DXlist = list()
Braaklist = list()
CDRSBlist = list()
for st in SID:
    DXlist.append(Mapping_SID_DX[st])
    Braaklist.append(Mapping_SID_Braak[st])
    CDRSBlist.append(Mapping_SID_CDRSB[st])

In [16]:
adata_adni.obs['DX'] = DXlist
adata_adni.obs['braaksc_trans'] = Braaklist
adata_adni.obs['CDRSOB'] = CDRSBlist

In [17]:
adata_adni.obs

,DX,braaksc_trans,CDRSOB
DCR00001_1,AD,5,5.0
DCR00004_1,MCI,5,0.5
DCR00009_1,CTL,5,0.0
DCR00013_1,CTL,5,0.0
DCR00014_1,CTL,5,inf
...,...,...,...
TLSMCI607_1,MCI,5,1.0
TLSMCI608_1,MCI,5,1.0
TLSMCI609_1,MCI,5,0.5
TLSMCI611_1,MCI,5,1.0


# Classify

In [18]:
anndata_Target = adata_adni

In [19]:
anndata_Target.obs

,DX,braaksc_trans,CDRSOB
DCR00001_1,AD,5,5.0
DCR00004_1,MCI,5,0.5
DCR00009_1,CTL,5,0.0
DCR00013_1,CTL,5,0.0
DCR00014_1,CTL,5,inf
...,...,...,...
TLSMCI607_1,MCI,5,1.0
TLSMCI608_1,MCI,5,1.0
TLSMCI609_1,MCI,5,0.5
TLSMCI611_1,MCI,5,1.0


In [20]:
Label = list()
SID = list(anndata_Target.obs.index)

for gt in SID:
    CRD = anndata_Target.obs.loc[gt,:].CDRSOB
    Braak = anndata_Target.obs.loc[gt,:].braaksc_trans
    Dx = anndata_Target.obs.loc[gt,:].DX
    if CRD > 4 and Braak >=4:
        Label.append("Typical AD")
    elif CRD <= 4 and Braak >=4 and CRD > 0:
        Label.append("Asym AD")
    elif CRD > 4 and Braak <=2:
        Label.append("Low-NFT AD")
    elif CRD <= 0.5 and Braak <=2:
        Label.append("Control")
    else:
        Label.append("Others")

In [23]:
anndata_Target.obs['NewLabel'] = Label

In [24]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Asym AD']
counter = collections.Counter(df['DX'])
print(counter)
Acc_Asym = (counter['CTL'] + counter['MCI']) / df.shape[0]
print("Asym AD transfer accuracy is ", Acc_Asym)

Counter({'MCI': 200, 'AD': 49, 'CTL': 36})
Asym AD transfer accuracy is  0.8280701754385965


In [25]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Typical AD']
counter = collections.Counter(df['DX'])
print(counter)
Acc_TAD = (counter['AD'] + counter['MCI']) / df.shape[0]
print("Typical AD transfer accuracy is ", Acc_TAD)

Counter({'AD': 174, 'CTL': 20, 'MCI': 16})
Typical AD transfer accuracy is  0.9047619047619048


In [26]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Low-NFT AD']
counter = collections.Counter(df['DX'])
print(counter)
Acc_LTau = (counter['AD'] + counter['MCI']) / df.shape[0]
print("Low-NFT AD transfer accuracy is ", Acc_LTau)

Counter()


ZeroDivisionError: division by zero

In [27]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Control']
counter = collections.Counter(df['DX'])
print(counter)
Acc_CN = (counter['CTL']) / df.shape[0]
print("Control transfer accuracy is ", Acc_CN)

Counter()


ZeroDivisionError: division by zero

In [ ]:
df = anndata_Target.obs[anndata_Target.obs.NewLabel=='Others']
counter = collections.Counter(df['DX'])
print(counter)

In [ ]:
Acc_Avg = (Acc_Asym + Acc_CN + Acc_LTau + Acc_TAD)/4
print("Avg acc is ", Acc_Avg)

In [ ]:
counter = collections.Counter(Label)
print(counter)

In [ ]:
len(Label)-247

In [6]:
import collections
# For rosmap blood
Blood_ROSMAP_Info = Blood_Info
BloodSampleID_Rosmap = list(adata_rosmap.obs.index)

SID = list(Blood_ROSMAP_Info.specimenID)
ADType = list(Blood_ROSMAP_Info.Group1)
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
frequency_rosmap = collections.Counter(ADType)
print(dict(frequency_rosmap))

Rosmap_Sample_CN = list()
Rosmap_Sample_TAD = list()
Rosmap_Sample_AAD = list()
for i in range(len(SID)):
    if ADType[i] == 'Control':
        Rosmap_Sample_CN.append(SID[i])
    if ADType[i] == 'Typical AD' or ADType[i] == 'Low-NFT AD':
        Rosmap_Sample_TAD.append(SID[i])
    if ADType[i] == 'Asym AD':
        Rosmap_Sample_AAD.append(SID[i])
#print(Rosmap_Sample_CN)
#print(Rosmap_Sample_TAD)
#print(Rosmap_Sample_AAD)

SType_Blood_Rosmap = list()
for i in range(len(BloodSampleID_Rosmap)):
    if BloodSampleID_Rosmap[i] in SID:
        indt = SID.index(BloodSampleID_Rosmap[i])
        tmp = Mapping[ADType[indt]]
        SType_Blood_Rosmap.append(tmp)
print(SType_Blood_Rosmap)
print(len(SType_Blood_Rosmap))

{'Typical AD': 46, 'Asym AD': 88, 'Control': 18, 'Low-NFT AD': 5}
[1, 2, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 2, 3, 2, 1, 0, 2, 2, 3, 1, 0, 0, 2, 2, 2, 2, 3, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 2, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2]
157


In [7]:
Adni_SID = list(Blood_Adni_Info.PTID)
Adni_Type = list(Blood_Adni_Info.DX)

SID_data = list(adata_adni.obs.index)

Adni_Type_Data = list()
for i in range(len(SID_data)):
    if SID_data[i] in Adni_SID:
        indt = Adni_SID.index(SID_data[i])
        tmp = Adni_Type[indt]
        Adni_Type_Data.append(tmp)
print(Adni_Type_Data)

['CN', 'CN', 'Dementia', 'CN', 'MCI', 'MCI', 'CN', 'MCI', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'Dementia', 'Dementia', 'MCI', 'MCI', 'MCI', 'Dementia', 'MCI', 'MCI', 'Dementia', 'MCI', 'MCI', 'Dementia', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'Dementia', 'CN', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'MCI', 'MCI', 'CN', 'Dementia', 'CN', 'CN', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'Dementia', 'CN', 'MCI', 'Dementia', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'MCI', 'CN', 'MCI', 'CN', 'Dementia', 'MCI', 'MCI', 'MCI', 'MCI', 'CN', 'Dementia', 'MCI', 'MCI', 'MCI', 'MCI', 'Dementia', 'Dementia', 'MCI', 'CN', 'Dementia', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'MCI', 'MCI', 'CN', 'MCI', 'MCI', 'MCI', 'Dementia', 'MCI', 'CN', 'MCI', 'MCI', 'MCI', 'CN', 'CN', 'CN', 'MCI', 'Dementia', 'CN', 'MCI', 'MCI', 'CN', 'MC

In [8]:
# convert predited label
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
y_pred = np.zeros(len(SID_data))
for i in range(len(SID_data)):
    y_pred[i] = Mapping[Mapping_ADNIID_PreLabel[SID_data[i]]]
    

In [9]:
Mapping = {'CN': 0, 'Dementia': 1, 'MCI': 2}
y_true_lb = Adni_Type_Data
y_true = np.zeros(len(y_true_lb))
for i in range(len(y_true_lb)):
    y_true[i] = Mapping[y_true_lb[i]]

In [10]:
pred_ind_cn = np.where(y_pred==0)[0]
true_ind_cn = np.where(y_true==0)[0]
overlap = list(set(pred_ind_cn) & set(true_ind_cn))
if len(pred_ind_cn) == 0:
    print("precision of CN is 0")
else:
    print("precision of CN is ", len(overlap)/len(pred_ind_cn))

pred_ind_tad = np.where(y_pred==1)[0]
true_ind_tad = np.where(y_true==1)[0]
overlap = list(set(pred_ind_tad) & set(true_ind_tad))
print("precision of TAD is ", len(overlap)/len(pred_ind_tad))

pred_ind_aad = np.where(y_pred==2)[0]
true_ind_aad = np.where(y_true==2)[0]
overlap = list(set(pred_ind_aad) & set(true_ind_aad))
print("precision of AAD is ", len(overlap)/len(pred_ind_aad))

precision of CN is 0
precision of TAD is  0.15660919540229884
precision of AAD is  0.5208333333333334


In [11]:
pred_ind_cn = np.where(y_pred==0)[0]
true_ind_cn = np.where(y_true==0)[0]
overlap = list(set(pred_ind_cn) & set(true_ind_cn))
#if len(pred_ind_cn) == 0:
#    print("precision of CN is 0")
#else:
#    print("precision of CN is ", len(overlap)/len(pred_ind_cn))


pred_ind_aad = np.where(y_pred==2)[0]
true_ind_aad = np.where(y_true==2)[0]
overlap = list(set(pred_ind_aad) & set(true_ind_aad))
#print("precision of AAD is ", len(overlap)/len(pred_ind_aad))

pred_ind_tad = np.where(y_pred==1)[0]
true_ind_tad = np.where(y_true==1)[0]
true_ind_tad_aad = list(true_ind_aad) + list(true_ind_tad)
overlap = list(set(pred_ind_tad) & set(true_ind_tad_aad))
print("precision of TAD is ", len(overlap)/len(pred_ind_tad))

pred_ind_aad_cn = list(pred_ind_aad) #+ list(pred_ind_cn)
true_ind_aad_cn = list(true_ind_cn) + list(true_ind_aad)
overlap = list(set(pred_ind_aad_cn) & set(true_ind_aad_cn))
print("precision of AAD  is ", len(overlap)/len(pred_ind_aad_cn))

pred_ind_aad_cn = list(pred_ind_cn) #list(pred_ind_aad) + 
true_ind_aad_cn = list(true_ind_cn) + list(true_ind_aad)
overlap = list(set(pred_ind_aad_cn) & set(true_ind_aad_cn))
if len(pred_ind_aad_cn) == 0:
    print("precision of CN  is 0")
else:
    print("precision of CN  is ", len(overlap)/len(pred_ind_aad_cn))

pred_ind_lt = np.where(y_pred==3)[0]
true_ind_lt = np.where(y_true==1)[0]
true_ind_lt_aad = list(true_ind_aad) + list(true_ind_lt)
overlap = list(set(pred_ind_lt) & set(true_ind_lt_aad))
if len(pred_ind_lt) == 0:
    print("precision of LowTau is 0")
else:
    print("precision of LowTau is ", len(overlap)/len(pred_ind_lt))

precision of TAD is  0.6695402298850575
precision of AAD  is  0.8541666666666666
precision of CN  is 0
precision of LowTau is 0
